In [6]:
#empty cache just in case
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -U bitsandbytes

Looking in indexes: https://download.pytorch.org/whl/cu118


In [7]:

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers peft accelerate datasets


Looking in indexes: https://download.pytorch.org/whl/cu121


In [8]:
!pip install -U transformers==4.44.2
# !pip install -U bitsandbytes==0.43.3
!pip install -U accelerate peft
# import bitsandbytes as bnb
# print(bnb.__version__)



In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer , BitsAndBytesConfig
import torch

torch.cuda.empty_cache()

model_name = "Qwen/Qwen2.5-1.5B-Instruct"
#configuration for quantizing the qwen model (windows setup)
# bnb_cfg = BitsAndBytesConfig(
#     load_in_4bit=True,                # 4-bit
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",        
#     bnb_4bit_compute_dtype=torch.bfloat16 
# )

#linux quantization:
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,                
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",        
    bnb_4bit_compute_dtype=torch.bfloat16 
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_cfg,
    #torch_dtype=torch.float16,   
    quantization_config= bnb_cfg ,   
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [10]:
#simple adapter cache to fix later:
adapter_cache = {}
def get_adapter(adapter_name):
    if adapter_name in adapter_cache:
        return adapter_cache[adapter_name]
    model = load_adapter(f"./adapters/{adapter_name}")
    adapter_cache[adapter_name] = model
    if len(adapter_cache) > CACHE_LIMIT:
        evict_one()  # define later
    return model


In [11]:
#adapter from: https://huggingface.co/DreamGallery/Qwen-Qwen2.5-1.5B-Instruct-1727452927
# from peft import PeftModel

# model = PeftModel.from_pretrained(
#     base_model,
#     "DreamGallery/Qwen-Qwen2.5-1.5B-Instruct-1727452927",  # adapter path
#     trust_remote_code=True
# )


In [12]:
prompt = "Explain the difference between L1 and L2 regularization in machine learning."
inputs = tokenizer(prompt, return_tensors="pt").to(base_model.device)
with torch.no_grad():
    out = base_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))


2025-11-19 12:49:47.078174: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763556587.516775      39 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763556587.647859      39 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Explain the difference between L1 and L2 regularization in machine learning. L1 and L2 regularization are two common techniques used to prevent overfitting in machine learning models, but they work differently.

L1 regularization (also known as Lasso regression) adds a penalty term to the loss function that is proportional to the absolute value of the coefficients. This means that it encourages some of the


In [32]:
#dataset of prompts:
from datasets import load_dataset

ds1 = load_dataset("fka/awesome-chatgpt-prompts")

In [30]:
from datasets import load_dataset

ds = load_dataset("RUC-DataLab/DataScience-Instruct-500K")

In [24]:
from datasets import load_dataset

ds2 = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en")

In [33]:
import pandas as pd
ds = pd.DataFrame(ds)
print(ds.head())

                                               train
0  {'id': 0, 'messages': [{'content': '## Instruc...
1  {'id': 0, 'messages': [{'content': '## Instruc...
2  {'id': 0, 'messages': [{'content': '## Instruc...
3  {'id': 0, 'messages': [{'content': '## Instruc...
4  {'id': 0, 'messages': [{'content': '## Instruc...


In [ ]:
def tokenize_ds(batch):
    prompts = []
    targets = []

    for messages, evaluation in zip(batch["messages"], batch["evaluation"]):
        if isinstance(messages, list):
            prompt = "\n".join([f"{m.get('role', '')}: {m.get('content', '')}" for m in messages])
        else:
            prompt = str(messages)

        prompts.append(prompt)
        targets.append(str(evaluation))

    return tokenizer(
        prompts,
        text_target=targets,
        truncation=True,
        padding="max_length",
        max_length=512,
    )

def tokenize_ds1(batch):
    prompts = []
    targets = []

    for act, prompt in zip(batch["act"], batch["prompt"]):
        if isinstance(act, list):
            inst = " ".join(act)
        else:
            inst = str(act)

        prompts.append(inst)
        targets.append(str(prompt))

    return tokenizer(
        prompts,
        text_target=targets,
        truncation=True,
        padding="max_length",
        max_length=512,
    )
def tokenize_ds3(batch):
    return tokenizer(
        batch["Question"],
        text_target=batch["Response"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )



train1 = ds.map(tokenize_ds, batched=True, batch_size=256)
train2 = ds1.map(tokenize_ds1, batched=False)
# train3 = ds2.map(tokenize_ds2, batched=False)

In [ ]:
from peft import LoraConfig 
lora_cfg_1 = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.1,
    task_type="CAUSAL_LM"
)
lora_cfg_2 = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.1,
    task_type="CAUSAL_LM"
)


In [ ]:
# import bitsandbytes as bnb
# print(bnb.__version__)

print("CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.get_device_name(0))

In [ ]:
!nvidia-smi

In [ ]:
from peft import get_peft_model
from transformers import TrainingArguments, Trainer
torch.cuda.empty_cache()

model1 = get_peft_model(base_model, lora_cfg_1)
args1 = TrainingArguments(
    per_device_train_batch_size=1,
    num_train_epochs=1,
    learning_rate=2e-4,
    output_dir="./adapter1",
    gradient_accumulation_steps=1,
    logging_dir="./logs",         # where logs are saved
    logging_steps=5,              # print every 5 steps
    report_to="none",             # disable WandB, keep console output
    disable_tqdm=False            # show progress bar
)
small_train1 = train1["train"].select(range(100))  # use only first 100 samples for faster runtime 

trainer1 = Trainer(model=model1, args=args1, train_dataset=small_train1)
trainer1.train()
model1.save_pretrained("./adapters/adapter1")

del model1
torch.cuda.empty_cache()

model2 = get_peft_model(base_model, lora_cfg_2)
args2 = TrainingArguments(
    per_device_train_batch_size=1,
    num_train_epochs=1,
    learning_rate=2e-4,
    output_dir="./adapter2",
    gradient_accumulation_steps=1,
    logging_dir="./logs",         # where logs are saved
    logging_steps=5,              # print every 5 steps
    report_to="none",             # disable WandB, keep console output
    disable_tqdm=False            # show progress bar
)
small_train2 = train2["train"].select(range(100))
trainer2 = Trainer(model=model2, args=args2, train_dataset=small_train2)
trainer2.train()
model2.save_pretrained("./adapters/adapter2")


#adapter 3:

model3 = get_peft_model(base_model, lora_cfg_2)
args3 = TrainingArguments(
    per_device_train_batch_size=1,
    num_train_epochs=1,
    learning_rate=2e-4,
    output_dir="./adapter3",
    gradient_accumulation_steps=1,
    logging_dir="./logs",         # where logs are saved
    logging_steps=5,              # print every 5 steps
    report_to="none",             # disable WandB, keep console output
    disable_tqdm=False            # show progress bar
)
small_train3 = train3["train"].select(range(100))
trainer3 = Trainer(model=model3, args=args3, train_dataset=small_train3)
trainer3.train()
model3.save_pretrained("./adapters/adapter3")


In [ ]:
from peft import PeftModel
#model 2 is on GPU:
print(next(model2.parameters()).device)
#model 1 is on GPU:
model1_loaded = PeftModel.from_pretrained(base_model, "./adapters/adapter1")
print(next(model1_loaded.parameters()).device)


In [ ]:
from transformers import  AutoTokenizer
from peft import PeftModel

model_name = "Qwen/Qwen2.5-1.5B-Instruct"
adapter_path = "./adapters/adapter1"

tokenizer = AutoTokenizer.from_pretrained(model_name)


model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()

text ="""### Instruction:
explain the difference between L1 and L2 regularization in machine learning.

### Input:

### Response:"""
 
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=150, temperature=0.7, top_p=0.9)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded)

In [ ]:
!git clone https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2


In [ ]:

from sentence_transformers import SentenceTransformer , util

model = SentenceTransformer("./all-MiniLM-L6-v2")
adapter_tasks = [
    "role-playing instruction following across many creative and utility roles",
    "data science and machine learning problem solving with step-by-step reasoning",
    "medical diagnosis and clinical reasoning with detailed chain-of-thought explanations",
    "summarization of long articles"
]
adapter_embs = model.encode(adapter_tasks)

print(adapter_embs)
def select_adapter(user_prompt):
    prompt_emb = model.encode([user_prompt], normalize_embeddings=True)[0]
    sims = util.cos_sim(prompt_emb, adapter_embs)[0]
    top = sims.argmax().item()
    best_score = sims[top].item()
    return adapter_tasks[top], best_score
task, score = select_adapter("Can you summarize this text?")
print(task, score)


In [ ]:
prompt = "Explain how blood pressure works in the human body."
adapter, scores = select_adapter(prompt)
print(adapter)   
print(scores)    
